## Search and Optimisation - Lab Sessions
### CCE2503
####  developed by - Adrian Muscat, 2022, 2023
---


In this notebook you will use or implement the following
1. Interval halving algorithm
1. Golden search method
2. Finding an initial bracket for line searches
3. Basic gradient descent algorithm
4. Accelerating GD with momentum
4. Timing of python code
5. Counting the number of times a function is called.


In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib inline

### Interval halving algorithm
Below is an implementation of the interval halving algorithm

In [ ]:
def interval_halving_algo(cost_func, a=0, b=1, e = 0.001, max_itr=100):
    """
    cost_func:  function that computes cost or loss
    [a,b] : Intial interval of uncertainty
    e : target interval of uncertainty
    max_itr: maximum number of iterations
    returns:
    [a,b]: Final interval of uncertainty
    n : number of iterations
    """
    #
    n=0
    D = b - a  #Interval of uncertainty
    x_m = a + D/2
    y_m = cost_func(x_m)        
    #
    while D>e and n<max_itr:
        n += 1
        x_l = a + D/4
        y_l = cost_func(x_l)
        x_u = b - D/4
        y_u = cost_func(x_u)
        if y_l < y_m:
            b = x_m
            x_m = x_l
            y_m = y_l
            D = b - a
        elif y_u < y_m:
            a = x_m
            x_m = x_u
            y_m = y_u
            D = b - a
        else:
            a = x_l
            b = x_u
            D = b - a
    #
    return a, b, n

### Let us define and plot some loss functions

In [ ]:
def squaring_function(z):
    """
    uni-variate squaring function, minimum is at z=1.5
    z: input values, vector of dimension n
    returns:
    value of function for each input element in n dimensions
    """
    return (z-1.3)**2.

def weighted_exponential_function(z):
    """
    uni-variate exponential function, minimum is at z=
    z: input values, vector of dimension n
    returns:
    value of function for each input element in n dimensions
    """
    return -z*np.exp(-z)

def shifted_exponential_function(z):
    """
    uni-variate exponential function, minimum is at z=
    z: input values, vector of dimension n
    returns:
    value of function for each input element in n dimensions
    """
    return 2 - 4*(z+1) + np.exp(z+1)

### Plot the above three functions

In [ ]:
f = [squaring_function, 
     weighted_exponential_function, 
     shifted_exponential_function]

x = np.linspace(-3,3,50)

for item in f:
    y = item(x)
    plt.plot(x,y)
plt.legend(f)
plt.axis([-3,3,-1,1])
plt.show()

### Use the interval-halving algorithm to find the minimum of the squaring function. You can start with the interval [-3, 3]

In [ ]:
print("weighted_exp_function: ",interval_halving_algo(weighted_exponential_function, 
                      a=0, b=2, e=0.0001, max_itr=100))
print("squaring_function: ",interval_halving_algo(squaring_function, 
                      a=0, b=2, e=0.0001, max_itr=100))

### Below is an implementation for initial bracket search

In [ ]:
def get_initial_bracket(f, x=0, s=1e-2, k=2.0):
    """
    Find an interval [a,b] along a coordinate direction in which a minimum exists
    f : multivariate function where minimum exists (n-dimensional)
    x : starting position  (vector in n-dimensions)
    s : initial step size (scalar)
    k : interval expanding factor (scalar)
    returns:
    [a,b] : return scalar quantities a, b (bracket or interval limits)
    """
    a = x
    ya = f(x)
    b = a + s
    x=b
    yb = f(x) 
    if yb > ya:
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    flag=True
    while flag:
        c = b + s
        x = c
        yc = f(x)
        if yc > yb:
            flag = False
        else:
            a, ya, b, yb = b, yb, c, yc # comment this line to preserve starting point
            s *= k
    if a < c: a_b = [a, c] 
    else: a_b=[c, a]
    return a_b

### Use the  get_initial_bracket() function to find an initial bracket and then proceed with the interval reduction algorithm

In [ ]:
a,b = get_initial_bracket(squaring_function, x=-1, s=.1, k=1.1 )
print(a,b)

In [ ]:

x1,x2,n = interval_halving_algo(squaring_function, 
                      a, b, e=0.0001, max_itr=100)
print(x1,x2,n)


### Timing the code

In [ ]:
N=1000
time_start = time.time()
for n in range(N):
    r = interval_halving_algo(squaring_function, 
                      a=0, b=2, e=0.0001, max_itr=100)
time_end = time.time()
print("Computational time: ",(time_end - time_start)/N)
print("squaring_function: ",r)

In [ ]:
from timeit import default_timer as timer
N=1000
start = timer()

for n in range(N):
    r = interval_halving_algo(squaring_function, 
                      a=0, b=2, e=0.0001, max_itr=100)

end = timer()
print((end - start)/N)

In [ ]:
%timeit max(range(100000))

### Let's count the number of function calls
### One neat method is based on defining a wrapper function and casting it in a class 

In [ ]:
class function_call_count:
    def __init__(self, name, count):
        self.name = name
        self.count = count

    def display_count(self): # Method to display count
        print("Function call count is ", self.count)
        
    def reset_count(self):  # Method to reset count
        self.count = 0
    
    def call_func(self,func):  # func is the function to be decorated
                               # call_func is the decorator function
        def wrapper(*args, **kwargs):
            self.count += 1  # increment count when func is called
            result = func(*args, **kwargs)
            return result
        return wrapper



In [ ]:
# We want to count the number of times the loss function 
# is called when using the interval-halving algorithm
#
# First we create an instance of class function_call_count
p1 = function_call_count(name="Interval-Halving", count=0)
#
# Second, we decorate the squaring function
func_1 = p1.call_func(squaring_function)
#
# Third, we call the interval-halving algorithm 
# specifying the decorated loss function
p1.reset_count()  
final_interval = interval_halving_algo(func_1, a=-3, b=3, 
                      e=0.001, max_itr=100)
p1.display_count()
print('Final interval and iterations',final_interval)

### Include the initial bracket algorithm. What is the new count?

In [ ]:
p1.reset_count()  # reset the counter to zero

initial_interval = get_initial_bracket(func_1, 
                                       x=-1, s=.1, k=2 )
print(initial_interval)
p1.display_count()

final_interval = interval_halving_algo(func_1, 
                                       a = initial_interval[0],
                                       b = initial_interval[1], 
                                       e = 0.001, max_itr=100)


p1.display_count()

print('Final interval and iterations',final_interval)

### Implement the Golden Search Algorithm

### Compare computational time and number of function evaluations for the two interval reduction methods

In [ ]:
...


### We will now consider a two dimensional loss function

In [ ]:
def rosenbrock(x, a=1, b=5):
    """
    Rosenbrock's function is a 2D uni-modal function
    This implementation follows the definition in Kochenderfer & Wheeler, 2019
    x : x is a numpy array of dimensions [2,m], where m is the number of 2D points
    """
    return (a-x[0])**2 + b*(x[1] - x[0]**2)**2

### Plot the Rosenbrock function as a contour plot and 3D wire-mesh

In [ ]:
N=80
limit=2
xx = np.linspace(-limit, limit, N)
yy = np.linspace(-limit, limit, N)
X = np.repeat(xx,N).reshape(N,N)
Y = np.tile(yy,N).reshape(N,N)
#
x = np.column_stack((X.flatten(),Y.flatten())).T
Z = rosenbrock(x).reshape(N,N)
#
plt.contour(X,Y,Z, levels=100)

In [ ]:
# Plot a wireframe.
#
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(X, Y, np.log(Z), rstride=2, cstride=2)
ax.view_init(10, 60) # elevation, azimuth
ax.view_init(0, 90) # elevation, azimuth

### Implement the Coodinate Descent algorithm to find the location of the minimum in the rosenbrock function

In [ ]:
...

### Implement Gradient Descent algorithm to find the location of the minimum in the rosenbrock function

In [ ]:
...

### Compare the number of iterations and computational time taken for the Coodinate Descent and Gradient Descent algorithms.

In [ ]:
...